In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from time import sleep
import minumbers as minu
import multiprocessing 

In [ ]:
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage{xcolor}')
inf = np.inf
pi = np.pi
rand = np.random.random
deg = np.degrees
rad = np.radians
Layout = widgets.Layout
%matplotlib widget

In [ ]:
%%html       
<style>
    .cplx_rat_exp{
        background-color: #AAA;
        color: #FFF;
        font-weight: bold;
        text-align: center;
    }
    .shadow{
        box-shadow: 8px 8px 10px #444;
    }

</style>

In [ ]:
def draw_cplx_root(fig, amount=1, angle=42, alpha_num=1, alpha_den=7, speed=0):

    def draw(pause):
        fig.clf()
        ax1, ax2 = fig.subplots(2, 1, gridspec_kw={
                                'height_ratios': [ax_ratio, 1], 'hspace': 0})
        ax2.axis('off')
        ax1.set_facecolor('#FAFAFA')
        ax1.spines['left'].set_position('zero')
        ax1.spines['bottom'].set_position('zero')
        ax1.spines['right'].set_color('none')
        ax1.spines['top'].set_color('none')
        ax1.spines['left'].set_zorder(2)
        ax1.spines['bottom'].set_zorder(2)
        ax1.spines['right'].set_zorder(2)
        ax1.spines['top'].set_zorder(2)
        ax1.plot(1, 0, ">k", transform=ax1.get_yaxis_transform(), clip_on=False)
        ax1.plot(0, 1, "^k", transform=ax1.get_xaxis_transform(), clip_on=False)

        r_max = np.max([r_num, r_base])
        ax1.set_xlim([-1.1*r_max, +1.1*r_max])
        ax1.set_ylim([-1.1*r_max, +1.1*r_max])
        ax1.set_xlabel('Re', loc='right') 
        ax1.set_ylabel("Im", loc='top', rotation=0)
        ax1.grid()
        ax1.plot(0, 0, 'ok', zorder=7)
        fig.canvas.draw_idle()
        fig.canvas.flush_events()

        arrowprops_vec = {'shrink': 0, 'width': 1, 'headwidth': 5,
                          'headlength': 5, 'connectionstyle': "arc3"}
        arrowprops_line = {'shrink': 0, 'width': 1, 'headwidth': 0,
                           'headlength': .5, 'connectionstyle': "arc3"}

        phi_z = pot[0][1]
        x = (np.cos(phi_z)+1)*.5
        y = (np.sin(phi_z)+1)*.5

        zz = patches.FancyArrowPatch((.5, .5), (x, y),
                                         lw=5,
                                         color="#0f0",
                                         zorder = 3,
                                         transform=ax1.transAxes)
        ax1.add_patch(zz)
        for z in pot:
            x = z[0]*np.cos(z[1])
            y = z[0]*np.sin(z[1])
            ax1.annotate("",xy=(x, y), xytext=(0, 0),
                               arrowprops={'color': '#F00', **arrowprops_vec})
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
            sleep(pause)
            
        for i in range(len(pot)-1):
            ax1.texts[-2].remove()
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
            sleep(pause)

        ax1.texts[-1].remove()
        for i in range(1, alpha_den+1):
            r = r_num**(1/i)
            x = r * np.cos(num_phi)
            y = r * np.sin(num_phi)
            ax1.annotate("", xy=(x, y), xytext=(0, 0),
                               arrowprops={'color': "#F00", **arrowprops_vec})
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
            sleep(pause)
            ax1.texts[-1].remove()
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
        
        mm = patches.FancyArrowPatch((0, 0), (x*1.05, y*1.05),
                                         lw=4, color="#f00",
                                         zorder = 4,
                                         )
        ax1.add_patch(mm)
    
        n = 10
        for i in range(1, n+1):
            r_zoom = r_max - (r_max-r)*i/n
            ax1.set_xlim([-1.1*r_zoom, +1.1*r_zoom])
            ax1.set_ylim([-1.1*r_zoom, +1.1*r_zoom])
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
            sleep(pause)
        
        circle = patches.Circle([0, 0], r, fill=False)
        ax1.add_patch(circle)
        fig.canvas.draw_idle()
        fig.canvas.flush_events()

        for i in back_angles:
            x = r * np.cos(i)
            y = r * np.sin(i)
            ax1.plot([0, x*1.05], [0, y*1.05], zorder=1, color="#AAF", lw=5, alpha=.5)
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
            sleep(pause)
        
        for r_phi in root_phi:
            x = r * np.cos(r_phi)
            y = r * np.sin(r_phi)
            ax1.annotate("", xy=(x, y), xytext=(0, 0), zorder=6,
                               arrowprops={**arrowprops_vec, 'color': "#FA0"})
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
            sleep(pause)
        
        
        tax = ax2
        prec = 2
        deg_prec = 1
        phi0 = root_phi[0]
        phi0_deg = round(deg(phi0), deg_prec)
        d_phi = root_phi[1]-root_phi[0]
        d_phi_deg = round(deg(d_phi), deg_prec)
        ylim = tax.get_ylim()[1]

        w0 = minu.Complex(f"{r_pot:g}*!{root_phi[0]}")
        w1 = minu.Complex(f"{r_pot:g}*!{root_phi[1]}")
        wn_1 = minu.Complex(f"{r_pot:g}*!{root_phi[-2]}")
        wn = minu.Complex(f"{r_pot:g}*!{root_phi[-1]}")
        w01_latex = r"$w_{0}=" + f"{w0.get_latex(prec=prec, deg_prec=deg_prec)[1][1:-1]}"\
                    r",\quad w_{1}=" + f"{w1.get_latex(prec=prec, deg_prec=deg_prec, plus=phi0)[1][1:-1]}"
        wn_latex = None
        if alpha_den > 4:
            w01_latex += r"\,,\,\mathbf{...}$"
        else:
            w01_latex += r"$"
            
        if alpha_den > 4:
            wn_latex = r"$\mathbf...\,, w_{" + f"{alpha_den-2}" + r"}="\
                       f"{wn_1.get_latex(prec=prec, deg_prec=deg_prec, plus=phi0)[1][1:-1]}"\
                       r",\quad w_{" + f"{alpha_den-1}" + r"}="\
                       f"{wn.get_latex(prec=prec, deg_prec=deg_prec, plus=phi0)[1][1:]}"
        elif alpha_den == 4:
            wn_latex = r"$w_{2}="\
                       f"{wn_1.get_latex(prec=prec, deg_prec=deg_prec, plus=phi0)[1][1:-1]}"\
                       r",\quad w_{3}="\
                       f"{wn.get_latex(prec=prec, deg_prec=deg_prec, plus=phi0)[1][1:]}"
        elif alpha_den == 3:
            wn_latex = r"$w_{2}="\
                       f"{wn_1.get_latex(prec=prec, deg_prec=deg_prec, plus=phi0)[1][1:]}"

        tax.text(0, .7*ylim, r"$\varphi_{0}=" + f"{phi0_deg:g}" + r"^{\circ},\quad"
                 r"\Delta\varphi=" + f"{d_phi_deg:g}" + r"^{\circ}$")
        tax.text(0, .35*ylim, w01_latex)
        if wn_latex is not None:
            tax.text(0, .0*ylim, wn_latex)
            
        # ax1.text(0, 0, f"exectime:{time.time()-time0}")
        fig.canvas.draw_idle()
        fig.canvas.flush_events()

    alpha = alpha_num/alpha_den
    r_base = amount
    r_pot = r_base**alpha
    r_num = r_base**alpha_num
    phi = angle
    
    mod = 2 * pi
        
    pot = []
    for num in range(1, alpha_num + 1):
        pot.append([r_base**num, phi*num % mod])
    num_phi = pot[-1][-1]
    
    back_angles = []
    for den in range(0, alpha_den):
        back_angles.append(num_phi*(1-den/alpha_den))
        
    root_phi = []
    for k in range(alpha_den):
        root_phi.append(1/alpha_den*(num_phi + 2*k*np.pi))
        
    draw(speed)

In [ ]:
def observer(number):
    global draw_thread

    cplx = cplx_wdgt.complex_number
    rat = rat_wdgt.rational_number
    if None not in {cplx, rat}:
        if 'draw_thread' in globals():
            draw_thread.interrupt()

        draw_thread = minu.Interruptable_thread(
            draw_cplx_root,
            fig,
            cplx.amount,
            cplx.angle,
            rat.numerator,
            rat.denominator,
            speed_wdgt.value,
        )
        draw_thread.start()

def observer_proc(number):
    global draw_proc

    cplx = cplx_wdgt.complex_number
    rat = rat_wdgt.rational_number
    if None not in {cplx, rat}:
        if 'draw_proc' in globals():
            draw_proc.terminate()

        draw_proc = multiprocessing.Process(target=draw_cplx_root, args=(
                                            fig,
                                            cplx.amount,
                                            cplx.angle,
                                            rat.numerator,
                                            rat.denominator,
                                            speed_wdgt.value,
                                           ))
        draw_proc.start()


x_inchs = 5
ax_ratio = 8
figsize = [x_inchs, x_inchs*((ax_ratio+1)/ax_ratio)]
plt.close('all')
plt.ioff()
fig = plt.figure(figsize=figsize)
fig.canvas.toolbar_visible = False
plt.ion()
heading_wdgt = widgets.HTMLMath(r"Complex number $\mathbf{\large\color{lightgreen}{z}}$ with rational exponent"
                                r"$\mathbf{\large\quad\color{white}{\alpha=\frac{\color{red}{m}}{\color{orange}{n}}:}}$"
                                r" $\quad"
                                r"\mathbf{\large\color{white}{\color{orange}{w_{k}}=\color{lightgreen}{z}^\alpha="
                                r"\color{lightgreen}{z}^\frac{\color{red}{m}}{\color{orange}{n}}=\sqrt[\color{orange}{n}]{"
                                r"\color{lightgreen}{z}^\color{red}{m}}}}$",
                                layout=Layout(width="100%", padding="5px", margin="0px"))
heading_wdgt.add_class('cplx_rat_exp')
rat_wdgt = minu.RationalWidget(r"Exponent: $\alpha=\frac{m}{n}$", "12/7", [1, np.inf], [2, np.inf], shorten=None)
cplx_wdgt = minu.ComplexWidget("Complex number z", "1.23*!rad(23)")
speed_wdgt = widgets.FloatLogSlider(
    value=.01,
    base=10,
    min=-2, # max exponent of base
    max=1, # min exponent of base
    step=0.05, # exponent step
    description='Pause',
    width="100%"
)
speed_wdgt.add_class('cplx_rat_exp')

input_wdgts = widgets.VBox([rat_wdgt.get_widget("100%"), cplx_wdgt.get_widget("100%")],
                          layout=Layout(width="100%"))
left_wdgt = widgets.VBox([heading_wdgt, input_wdgts, speed_wdgt], layout=Layout(width="30%"))
wdgt = widgets.HBox([left_wdgt, fig.canvas], layout=Layout(width="80%", align_items="center"))

left_wdgt.add_class('shadow')

rat_wdgt.observe(observer)
cplx_wdgt.observe(observer)
speed_wdgt.observe(observer)
#wdgt.add_class('shadow')
display(wdgt)
fig.canvas.draw_idle()
observer("dummy")